In [43]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

import warnings
warnings.filterwarnings('ignore')

tensor([0.], device='cuda:0')
2.1.0
graph-tool version: 2.46


In [44]:
with open('../g_nx.pickle', 'rb') as f:
    g_nx = pickle.load(f)

In [45]:
# create a dictionary to store the attribute values
attr_dict = {}
for node in g_nx.nodes():
    attr = g_nx.nodes[node]['uri']
    if attr in attr_dict:
        attr_dict[attr].append(node)
    else:
        attr_dict[attr] = [node]
        
del attr_dict['']
len(attr_dict)

35042

In [42]:
keys_with_duplicates = [key for key, values in attr_dict.items() if len(values) > 1]

# print the keys with duplicates
print(len(keys_with_duplicates))

524


In [46]:
keys_with_duplicates[0]

'spotify:track:0WqIKmW4BTrj3eJFmnCKMv'

In [47]:
attr_dict['spotify:track:0WqIKmW4BTrj3eJFmnCKMv']

[2001, 38488]

In [ ]:
with open('../datasets_deepsnap_dict.pkl', 'rb') as f:
    # load the pickled dictionary from the file
    datasets = pickle.load(f)
    
ds_graph = Graph(g_nx)

task = 'link_pred'
dataset = GraphDataset([ds_graph], task=task, edge_train_mode='disjoint')
dataset_train, dataset_val, dataset_test = dataset.split(transductive=True, split_ratio=[0.8, 0.1, 0.1])
num_test_edges = dataset_test[0].edge_label_index.shape[1]

print("Test set has {} supervision (positive) edges".format(num_test_edges // 2))
print("Test set has {} message passing edges".format(dataset_test[0].edge_index.shape[1]))+

In [ ]:
class LightGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(LightGCNConv, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

    def forward(self, x, edge_index, size = None):
        out = self.propagate(edge_index, x=(x, x))
        return out

    def message(self, x_j):
        out = x_j
        return out

    def aggregate(self, inputs, index, dim_size = None):
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, dim=node_dim, reduce='mean')
        return out

In [ ]:
class LightGCN(torch.nn.Module):
    def __init__(self, train_data, num_layers, emb_size=16, initialize_with_words=False):
        super(LightGCN, self).__init__()
        self.convs = nn.ModuleList()
        assert (num_layers >= 1), 'Number of layers is not >=1'
        for l in range(num_layers):
            self.convs.append(LightGCNConv(input_dim, input_dim))

        # Initialize using custom embeddings if provided
        num_nodes = train_data.node_label_index.size()[0]
        self.embeddings = nn.Embedding(num_nodes, emb_size)
        if initialize_with_words:
            self.embeddings.weight.data.copy_(train_datanode_features)
        
        self.loss_fn = nn.BCELoss()
        self.num_layers = num_layers
        self.emb_size = emb_size
        self.num_modes = num_nodes

    def forward(self, data):
        edge_index, edge_label_index, node_label_index = data.edge_index, data.edge_label_index, data.node_label_index
        layer_embeddings = []
        
        x = self.embeddings(node_label_index)
        mean_layer = x

        # We take an average of ever layer's node embeddings
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            # print("x shape",x.shape)
            # print("mean_layer shape",mean_layer.shape)
            mean_layer += x

        mean_layer /= 4

        # Prediction head is simply dot product
        nodes_first = torch.index_select(x, 0, edge_label_index[0,:].long())
        nodes_second = torch.index_select(x, 0, edge_label_index[1,:].long())

        # Since we don't want a rank output, we create a sigmoid of the dot product
        out = torch.sum(nodes_first * nodes_second, dim=-1) # FOR RANKING
        pred = torch.sigmoid(out)

        return torch.flatten(pred)

    def loss(self, pred, label):
        return self.loss_fn(pred, label)

In [ ]:
best_model

In [ ]:
# args = {
#     'device' : 'cuda' if torch.cuda.is_available() else 'cpu',
#     'num_layers' : 3,
#     'emb_size' : 32,
#     'weight_decay': 1e-5,
#     'lr': 0.01,
#     'epochs': 600
# }

# datasets = {
#     'train': dataset_train[0],
#     'val': dataset_val[0],
#     'test': dataset_test[0]
# }
            
# input_dim = datasets['train'].num_node_features
# print(input_dim, args)

In [ ]:
best_model = torch.load('../best_lgcn.pkl')

def test(model, mode, args):
    model.eval()
    score = 0
    loss_score = 0

    data = datasets[mode]
    data.to(args["device"])

    pred = model(data)
    loss = model.loss(pred, data.edge_label.type(pred.dtype))
    score += roc_auc_score(data.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
    loss_score += loss.item()

    return pred, score, loss_score

preds_test, best_test_roc, test_loss = test(best_model, 'test', args)

In [ ]:
ds = datasets['test']
ds

In [ ]:
dict1 = {'preds':pred.to('cpu').detach().numpy(), 'edge_label': ds.edge_label.to('cpu').detach().numpy(), 'src_node': ds.edge_label_index[0].to('cpu').detach().numpy(), 'dest_node': ds.edge_label_index[1].to('cpu').detach().numpy()}
df = pd.DataFrame(dict1)

df_1 = df[df['edge_label'] == 1]
df_1

TODO: new playlist inference:

- if new playlist node under consideration is from the existing graph:
    - make bipartite graph, where one side is the playlist node and the other side are all track nodes except the ones the playlist under consideration is already connected to
    - run model on above graph and get top results.
    
- if new playlist does not exist in the existing graph (caveat: but some of its track nodes do):
    - add new nodes from the new playlist network (minus overlapping track nodes) to the existing, main graph
    - randomly initialize embeddings for the new nodes
    - figure out how to perform message passing and aggregation only on the new nodes and only update their (initially randomly initialized) embeddings
      ^^train over new nodes (freeze existing nodes updation) to get updated, better embeddings of the new nodes.
    - once the above is done, proceed with bipartite graph with new playlist node at one end and all other non-connected track nodes at the other.
    - run model on above graph and get top results.